In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
from test_utils import test

# Cross-Validation and Bias-Variance decomposition
## Cross-Validation
Implementing 4-fold cross-validation below:

In [2]:
from helpers import load_data

# load dataset
x, y = load_data()

In [3]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold.

    Args:
        y:      shape=(N,)
        k_fold: K in K-fold, i.e. the fold num
        seed:   the random seed

    Returns:
        A 2D array of shape=(k_fold, N/k_fold) that indicates the data indices for each fold

    >>> build_k_indices(np.array([1., 2., 3., 4.]), 2, 1)
    array([[3, 2],
           [0, 1]])
    """
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval : (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

In [4]:
test(build_k_indices)

✅ Your `build_k_indices` passes some basic tests.


For the following cross_validation( ) function you need to implement, you can help yourselves of the build_poly( ) and ridge_regression( ) functions that you implemented in lab 3. Copy paste the code in the build_polynomial.py and ridge_regression.py files, they should pass the two following tests.

In [5]:
from costs import compute_mse
from ridge_regression import ridge_regression
from build_polynomial import build_poly


test(build_poly)
test(ridge_regression)

❌ The are some issues with your implementation of `build_poly`:
**********************************************************************
File "/Users/cherie/Desktop/ML_course/labs/ex04/template/build_polynomial.py", line 17, in build_poly
Failed example:
    build_poly(np.array([0.0, 1.5]), 2)
Exception raised:
    Traceback (most recent call last):
      File "/opt/anaconda3/lib/python3.12/doctest.py", line 1368, in __run
        exec(compile(example.source, filename, "single",
      File "<doctest build_poly[0]>", line 1, in <module>
        build_poly(np.array([0.0, 1.5]), 2)
      File "/Users/cherie/Desktop/ML_course/labs/ex04/template/build_polynomial.py", line 27, in build_poly
        raise NotImplementedError
    NotImplementedError
**********************************************************************
❌ The are some issues with your implementation of `ridge_regression`:
**********************************************************************
File "/Users/cherie/Desktop/ML_course/

In [6]:
def compute_loss(y, tx, w, type='mse'):
    """Calculate the loss using either MSE or MAE.

    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        w: numpy array of shape=(2,). The vector of model parameters.

    Returns:
        the value of the loss (a scalar), corresponding to the input parameters w.
    """
    if type == 'mse':
        error_term = y - tx @ w
        divide_term = 1 / (2 * len(y))

        return divide_term * np.dot(error_term, error_term)
    else:
        divide_term = 1 / (len(y))
        return divide_term * np.sum(np.abs(y - tx @ w))

In [7]:
def cross_validation(y, x, k_indices, k, lambda_, degree):
    """return the loss of ridge regression for a fold corresponding to k_indices

    Args:
        y:          shape=(N,)
        x:          shape=(N,)
        k_indices:  2D array returned by build_k_indices()
        k:          scalar, the k-th fold (N.B.: not to confused with k_fold which is the fold nums)
        lambda_:    scalar, cf. ridge_regression()
        degree:     scalar, cf. build_poly()

    Returns:
        train and test root mean square errors rmse = sqrt(2 mse)

    >>> cross_validation(np.array([1.,2.,3.,4.]), np.array([6.,7.,8.,9.]), np.array([[3,2], [0,1]]), 1, 2, 3)
    (0.019866645527597114, 0.33555914361295175)
    """
 # Get k-th subgroup in test, the others in train
    test_indices = k_indices[k]
    train_indices = np.delete(k_indices, k, axis=0).flatten()
    
    x_train = x[train_indices]
    y_train = y[train_indices]
    x_test = x[test_indices]
    y_test = y[test_indices]
    
    # Form data with polynomial degree
    tx_train = build_poly(x_train, degree)
    tx_test = build_poly(x_test, degree)
    
    # Ridge regression
    weights = ridge_regression(y_train, tx_train, lambda_)
    
    # Calculate the loss for train and test data
    y_train_pred = np.dot(tx_train, weights.T)
    y_test_pred = np.dot(tx_test, weights.T)
    
    rmse_train = np.sqrt(2 * compute_loss(y_train, y_train_pred, weights , mse))
    rmse_test = np.sqrt(2 * compute_loss(y_test, y_test_pred, weights, mse))
    
    return rmse_train, rmse_test

In [8]:
# can lead to a numerical error if you use an older version than Python 3.9
test(cross_validation)

❌ The are some issues with your implementation of `cross_validation`:
**********************************************************************
File "__main__", line 15, in cross_validation
Failed example:
    cross_validation(np.array([1.,2.,3.,4.]), np.array([6.,7.,8.,9.]), np.array([[3,2], [0,1]]), 1, 2, 3)
Exception raised:
    Traceback (most recent call last):
      File "/opt/anaconda3/lib/python3.12/doctest.py", line 1368, in __run
        exec(compile(example.source, filename, "single",
      File "<doctest cross_validation[0]>", line 1, in <module>
        cross_validation(np.array([1.,2.,3.,4.]), np.array([6.,7.,8.,9.]), np.array([[3,2], [0,1]]), 1, 2, 3)
      File "/var/folders/0m/t98q9lwj7r59mrxwrbcg977h0000gn/T/ipykernel_74762/1708367960.py", line 28, in cross_validation
        tx_train = build_poly(x_train, degree)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^
      File "/Users/cherie/Desktop/ML_course/labs/ex04/template/build_polynomial.py", line 27, in build_poly
    

In [ ]:
from plots import cross_validation_visualization


def cross_validation_demo(degree, k_fold, lambdas):
    """cross validation over regularisation parameter lambda.

    Args:
        degree: integer, degree of the polynomial expansion
        k_fold: integer, the number of folds
        lambdas: shape = (p, ) where p is the number of values of lambda to test
    Returns:
        best_lambda : scalar, value of the best lambda
        best_rmse : scalar, the associated root mean squared error for the best lambda
    """

    seed = 12
    degree = degree
    k_fold = k_fold
    lambdas = lambdas
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss of training data and test data
    rmse_tr = []
    rmse_te = []
    best_rmse = float('inf')
    best_lambda = None

    # cross-validation over lambdas
    for lambda_ in lambdas:
        rmse_tr_tmp = []
        rmse_te_tmp = []
        
        # Perform cross-validation
        for k in range(k_fold):
            rmse_tr_k, rmse_te_k = cross_validation(y, x, k_indices, k, lambda_, degree)
            rmse_tr_tmp.append(rmse_tr_k)
            rmse_te_tmp.append(rmse_te_k)
        
        # Compute average RMSE for current lambda
        avg_rmse_tr = np.mean(rmse_tr_tmp)
        avg_rmse_te = np.mean(rmse_te_tmp)
        
        rmse_tr.append(avg_rmse_tr)
        rmse_te.append(avg_rmse_te)
        
        # Track the best lambda
        if avg_rmse_te < best_rmse:
            best_rmse = avg_rmse_te
            best_lambda = lambda_


    cross_validation_visualization(lambdas, rmse_tr, rmse_te)
    print(
        "For polynomial expansion up to degree %.f, the choice of lambda which leads to the best test rmse is %.5f with a test rmse of %.3f"
        % (degree, best_lambda, best_rmse)
    )
    return best_lambda, best_rmse


best_lambda, best_rmse = cross_validation_demo(7, 4, np.logspace(-4, 0, 30))

Your output should look like this for seed = 12, degree = 7 and k_fold = 4:

![alt text](cross_validation2.png)

You can play around with the number of folds and the degree of your polynomial expansion.

In [ ]:
best_lambda, best_rmse = cross_validation_demo(10, 4, np.logspace(-10, -2, 30))

In the previous task we did a grid search over several values of $\lambda$ for a fixed degree. We can also perform a grid search amongst $\lambda$ and degrees simultaneously:

In [ ]:
def best_degree_selection(degrees, k_fold, lambdas, seed=1):
    """cross validation over regularisation parameter lambda and degree.

    Args:
        degrees: shape = (d,), where d is the number of degrees to test
        k_fold: integer, the number of folds
        lambdas: shape = (p, ) where p is the number of values of lambda to test
    Returns:
        best_degree : integer, value of the best degree
        best_lambda : scalar, value of the best lambda
        best_rmse : value of the rmse for the couple (best_degree, best_lambda)

    >>> best_degree_selection(np.arange(2,11), 4, np.logspace(-4, 0, 30))
    (7, 0.004520353656360241, 0.28957280566456634)
    """

    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)

    best_rmse = float('inf')
    best_lambda = None
    best_degree = None

    # Loop over all degrees
    for degree in degrees:
        rmse_tr = []
        rmse_te = []

        # Loop over all lambdas for the current degree
        for lambda_ in lambdas:
            rmse_tr_tmp = []
            rmse_te_tmp = []
            
            # Perform k-fold cross-validation for each lambda and degree
            for k in range(k_fold):
                rmse_tr_k, rmse_te_k = cross_validation(y, x, k_indices, k, lambda_, degree)
                rmse_tr_tmp.append(rmse_tr_k)
                rmse_te_tmp.append(rmse_te_k)

            # Compute the average RMSE for this degree and lambda
            avg_rmse_tr = np.mean(rmse_tr_tmp)
            avg_rmse_te = np.mean(rmse_te_tmp)
            
            rmse_tr.append(avg_rmse_tr)
            rmse_te.append(avg_rmse_te)

            # Track the best lambda and degree based on test RMSE
            if avg_rmse_te < best_rmse:
                best_rmse = avg_rmse_te
                best_lambda = lambda_
                best_degree = degree

    return best_degree, best_lambda, best_rmse

In [ ]:
# can lead to a numerical error if you use an older version than Python 3.9
test(best_degree_selection)

best_degree, best_lambda, best_rmse = best_degree_selection(
    np.arange(2, 11), 4, np.logspace(-4, 0, 30)
)
print(
    "The best rmse of %.3f is obtained for a degree of %.f and a lambda of %.5f."
    % (best_rmse, best_degree, best_lambda)
)

## Bias-Variance Decomposition

In [ ]:
# true function we want to learn
def f_star(x):
    return x**3 - x**2 + 0.5


# plotting function for f_star
def plot_fstar(ax):
    xvals = np.arange(-1, 1, 0.01)
    ax.plot(xvals, f_star(xvals), linestyle="--", color="k", label="f_star")
    ax.set_ylim(-2, 2)

In [ ]:
# helper plot function
def plot_poly(x, y, weights, degree, ax, alpha=0.3):
    xvals = np.arange(-1, 1, 0.01)
    tx = build_poly(xvals, degree)
    f = tx.dot(weights)
    ax.plot(xvals, f, color="orange", alpha=alpha)
    ax.scatter(x, y, color="b", alpha=alpha, s=10)
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_title("Polynomial degree " + str(degree))
    ax.set_ylim(-1, 2)


# helper plot function
def plot_f(weights, degree, ax, label=None):
    xvals = np.arange(-1, 1, 0.01)
    tx = build_poly(xvals, degree)
    f = tx.dot(weights)
    ax.plot(xvals, f, color="black", alpha=1, label=label)
    ax.set_ylim(-1, 2)

Implement the following function: for 15 random datapoints, it finds the optimal fit (using the least square formula, with no regularisation λ) for a polynomial expansion of degree 1, 3 and 6.

In [ ]:
rom least_squares import least_squares


def bias_variance_one_seed(sigma, degrees, seed):
    """One run of the optimal fit for 15 random points and different polynomial expansion degrees.

    Args:
        sigma: scalar, noise variance
        degrees: shape = (3,), 3 different degrees to consider
        seed: integer, random see
    Returns:
    """

    # we will generate 15 random datapoints from the [-1, 1] uniform distribuion
    num_data = 15
    np.random.seed(seed)  # set random seed for reproducibility
    xs = np.random.uniform(-1, 1, num_data)
    # the outputs will be f_star(x) + some random gaussian noise of variance sigma**2
    ys = f_star(xs) + sigma * np.random.randn(num_data)

    fig, axs = plt.subplots(1, len(degrees), figsize=(20, 5))
    for index_degree, degree in enumerate(degrees):
        # Create polynomial features for the current degree
        x_poly = polynomial_features(xs, degree)

        # Fit the model using least squares
        weights, _ = least_squares(ys, x_poly)

        # Generate the prediction on a dense grid for smoother plotting
        x_dense = np.linspace(-1, 1, 100)
        x_poly_dense = polynomial_features(x_dense, degree)
        y_pred = x_poly_dense @ weights

        # Plot the true function, the data points, and the fitted polynomial
        axs[index_degree].scatter(xs, ys, color='b', label="Data points")
        axs[index_degree].plot(x_dense, y_pred, label=f"Degree {degree} fit")
        axs[index_degree].set_title(f"Degree {degree} fit")

        plot_fstar(axs[index_degree])
        axs[index_degree].legend()
    plt.show()


bias_variance_one_seed(0.1, [1, 3, 6], seed=2)

Your output should ressemble (for seed = 2) to this: 
![alt text](bias_variance_one_run.png)

Now to illustrate the bias variance tradeoff we will repeat many times the previous experiment but using a different random seed each time. We also plot (in plain black) the mean of all the orange functions obtained.

In [ ]:
def bias_variance_demo(sigma, degrees):
    """Illustration of the bias-variance tradeoff.

    Args:
        sigma: scalar, noise variance
        degrees: shape = (3,), 3 different degrees to consider
    Returns:
    """
    # define parameters
    seeds = range(400)  # number of runs
    num_data = 15

    fig, axs = plt.subplots(1, len(degrees), figsize=(20, 5))
    for index_degree, degree in enumerate(degrees):
          # Run the experiment over multiple seeds
        for seed in seeds:
            np.random.seed(seed)
            xs = np.random.uniform(-1, 1, num_data)
            ys = f_star(xs) + sigma * np.random.randn(num_data)

            # Create polynomial features and fit the model
            x_poly = build_poly(xs, degree)
            weights, _ = least_squares(ys, x_poly)

            # Generate predictions on a dense grid
            x_dense = np.linspace(-1, 1, 100)
            x_poly_dense = build_poly(x_dense, degree)
            y_pred = x_poly_dense @ weights


        plot_fstar(axs[index_degree])
        axs[index_degree].legend()
    plt.show()


bias_variance_demo(0.1, [1, 3, 6])

Your output should ressemble to this: 
![alt text](bias_variance.png)